In [1]:
import pyfits
import numpy as np
import scipy as sp
from matplotlib import pyplot as plt
%matplotlib inline

/u/ki/chto100/PhilProject/lsst/lib/python2.7/site-packages/matplotlib/__init__.py:816: UserWarning: Found matplotlib configuration in ~/.matplotlib/. To conform with the XDG base directory standard, this configuration location has been deprecated on Linux, and the new location is now /u/ki/chto100/.config/matplotlib/. Please move your configuration there to ensure that matplotlib will continue to find it in the future.
  _get_xdg_config_dir())


In [2]:
cat="/u/ki/chto100/code/redmapper_clf/data/redmapper_v6.4.16/y1a1_gold_1.0.3_wide+d10-mof-001b_run_redmapper_v6.4.16_lgt5_vl50_catalog.fit"
print cat[:-4]+"_members.fit"
realcat = pyfits.open("/u/ki/chto100/code/redmapper_clf/data/redmapper_v6.4.16/y1a1_gold_1.0.3_wide+d10-mof-001b_run_redmapper_v6.4.16_lgt5_vl50_catalog.fit")
realmem = pyfits.open("/u/ki/chto100/code/redmapper_clf/data/redmapper_v6.4.16/y1a1_gold_1.0.3_wide+d10-mof-001b_run_redmapper_v6.4.16_lgt5_vl50_catalog_members.fit")

/u/ki/chto100/code/redmapper_clf/data/redmapper_v6.4.16/y1a1_gold_1.0.3_wide+d10-mof-001b_run_redmapper_v6.4.16_lgt5_vl50_catalog_members.fit


(223636, 5)

In [23]:
cat=realcat[1].data
mem=realmem[1].data

In [24]:
lumbins = np.array(range(35))*0.2-25.
#lumbins = np.array(range(35))*0.1+15.
print lumbins

[-25.  -24.8 -24.6 -24.4 -24.2 -24.  -23.8 -23.6 -23.4 -23.2 -23.  -22.8
 -22.6 -22.4 -22.2 -22.  -21.8 -21.6 -21.4 -21.2 -21.  -20.8 -20.6 -20.4
 -20.2 -20.  -19.8 -19.6 -19.4 -19.2 -19.  -18.8 -18.6 -18.4 -18.2]


In [25]:
kcorr = pyfits.open("/nfs/slac/kipac/fs1/u/chto100/redmapper_clf/catche/DES_zlambda_0.3_new_y1a1_gold_1.0.3_wide+d10-mof-001b_run_redmapper_v6.4.16_lgt5_vl50real.fit")
kcorr = kcorr[0].data

In [26]:
mag = mem['REFMAG'] + (kcorr[:,2] - mem['model_mag'][:,3])
cengalindex = np.zeros_like(cat['id_cent'])
cenmag = np.zeros_like(cengalindex).astype(float)
        #Hash table taking galaxy ID to index
offset = np.min(mem['id'])
g_index = np.zeros(np.max(mem['id'])-offset+1)-1
g_index[mem['id']-offset] = np.array(range(len(mem)))
#modified by chto to handle that cat['id_cent']=0
for i in range(len(cengalindex[0])):
    index = np.where(cat['id_cent'][:,i]-offset>=0)
    cengalindex[index,i] = g_index[cat['id_cent'][index,i]-offset]
    cenmag[index,i] = mag[cengalindex[index,i]]

In [27]:
print (cat['p_cen'])

[[  9.91312802e-01   5.08883549e-03   2.95544486e-03   5.57411811e-04
    8.55670587e-05]
 [  9.99410510e-01   3.49117297e-04   1.94435415e-04   3.69092522e-05
    9.01986732e-06]
 [  7.48739421e-01   1.78796664e-01   6.19009137e-02   7.72900274e-03
    2.83397338e-03]
 ..., 
 [  5.93337715e-01   3.66590917e-01   1.98960137e-02   1.17906723e-02
    8.38465150e-03]
 [  8.09782386e-01   1.85155705e-01   2.91182380e-03   1.48393039e-03
    6.66124339e-04]
 [  8.11088383e-01   1.74758613e-01   7.06633087e-03   5.90787223e-03
    1.17881468e-03]]


In [28]:
cen_select=np.random.uniform(size=len(cat))
nclusters=10000
cat=cat[:nclusters]
cenmag=cenmag[:nclusters]

In [29]:
cen_select.shape

(223636,)

In [30]:
def count_galaxies_rand_cen(cat,cenmag,cengalindex,lumbins,weight_cen=0):
#    nclusters = len(cat)
#    nclusters=nclusters
    nlum = len(lumbins)
    dlum = lumbins[1]-lumbins[0]
    minlum = lumbins[0]-dlum/2.

    #Make the array that will contain the full count of galaxies
    #in each cluster
    count_arr = np.zeros([nclusters,nlum])

    #randoms for choosing which central
    #cen_select = np.random.uniform(size=nclusters)
    #Loop over central galaxies, counting each in turn
    ncen=1
    for i in range(nclusters):
        #If weighting, should check all good centrals
        if weight_cen==1:
            ncen = cat['ncent_good'][i]
        mybin = np.floor((cenmag[i,0:ncen]-minlum)/dlum)
        mybin = mybin.astype(int)
        myskip=False
        if ncen==0:
            myskip=True
        if ncen==1:
            if (mybin < 0) | (mybin >= nlum):
                myskip=True
        if ncen>1:
            count = 0
            for j in range(ncen):
                if (mybin[j] < 0) | (mybin[j] >= nlum):
                    count = count+1
            if count == ncen:
                myskip=True
            else:
                sublist = np.where( (mybin >= 0 ) & (mybin < nlum))[0]
                #print np.where( (mybin >= 0 ) & (mybin < nlum))

        if myskip:
            continue

        #If weighting, do a random selection for the central
        if i==11694:
            print mybin
        if cen_select[i] > np.sum(cat['p_cen'][i][0:ncen]):
            continue
        for j in range(ncen):
            if (cen_select[i] < np.sum(cat['p_cen'][i][0:j+1])) & (mybin[j] >= 0) & (mybin[j] < nlum):
                count_arr[i,mybin[j]] += 1.
                break

    return count_arr

In [31]:
#%time countarr=count_galaxies_rand_cen(cat,cenmag,cengalindex,lumbins,weight_cen=0)

In [41]:
def chto_count_galaxies_rand_cen(cat,cenmag,cengalindex,lumbins,weight_cen=0):
    nlum = len(lumbins)
#    nclusters = len(cat)
#    cen_select=np.random.uniform(size=len(cat))
    dlum = lumbins[1]-lumbins[0]
    minlum = lumbins[0]-dlum/2.
    chto_countArray=np.zeros([len(cat),nlum])
    cumCat_p=np.cumsum(cat['p_cen'],axis=1)
    mybin = np.floor((cenmag[:,:]-minlum)/dlum).astype(np.int)
    if weight_cen:
        ncen = np.repeat(cat['ncent_good'].reshape(-1,1),cat['p_cen'].shape[1],axis=1)
    else:
        ncen = np.repeat(np.ones(cat['p_cen'].shape[0]).reshape(-1,1),cat['p_cen'].shape[1],axis=1)
    
    cen_select2 = np.repeat(cen_select.reshape(-1,1),cat['p_cen'].shape[1],axis=1)
    ys = np.outer(np.ones(cat['p_cen'].shape[0]),np.arange(cat['p_cen'].shape[1]))

    mask = ((cumCat_p >= cen_select2)&(mybin>0) & (mybin<nlum)& (ys<ncen)).astype(np.float64)
    mask2 = np.cumsum(mask, axis=1)
    mask3 = np.cumsum(mask2, axis=1)
    mask3[np.where(mask3>1)]=0.
    newbin=np.sum((mybin*mask3), axis=1)
    mask_new=np.sum(mask3,axis=1)[:, np.newaxis]
    indexArray=np.hstack((np.arange(0,cat['p_cen'].shape[0]).reshape(-1,1),newbin.reshape(-1,1))).astype(np.int)
    chto_countArray[indexArray[:,0],indexArray[:,1]]=1.
    chto_countArray =  chto_countArray*mask_new
    return chto_countArray
def count_galaxies_p_cen(cat,cenmag,lumbins,weight_cen=0):
    nclusters = len(cat)

    nlum = len(lumbins)
    dlum = lumbins[1]-lumbins[0]
    minlum = lumbins[0]-dlum/2.

    #Make the array that will contain the full count of galaxies
    #in each cluster
    count_arr = np.zeros([nclusters,nlum])

    #Loop over central galaxies, counting each in turn
    ncen=1
    for i in range(nclusters):
        #If weighting, should check all good centrals
        if weight_cen==1:
            ncen = cat['ncent_good'][i]
        mybin = np.floor((cenmag[i,0:ncen]-minlum)/dlum)
        mybin = mybin.astype(int)
        myskip=False
        if ncen==0:
            myskip=True
        if ncen==1:
            if (mybin < 0) | (mybin >= nlum):
                myskip=True
        if ncen>1:
            count = 0
            for j in range(ncen):
                if (mybin[j] < 0) | (mybin[j] >= nlum):
                    count = count+1
            if count == ncen:
                myskip=True
            else:
                sublist = np.where( (mybin >= 0 ) & (mybin < nlum))[0]

        if myskip:
            continue

        if ncen==1:
            count_arr[i,mybin] += cat['p_cen'][i][0:ncen]
        else:
            for j in range(len(sublist)):
                count_arr[i,mybin[sublist[j]]] += cat['p_cen'][i][sublist[j]]
        #If not weighting, give best central all probability
        if weight_cen==0:
            count_arr[i,mybin] += cat['p_sat'][i][0:ncen]

    return count_arr
def chto_count_galaxies_p_cen(cat,cenmag,lumbins,weight_cen=0):
    nlum = len(lumbins)
#    nclusters = len(cat)
#    cen_select=np.random.uniform(size=len(cat))
    dlum = lumbins[1]-lumbins[0]
    minlum = lumbins[0]-dlum/2.
    chto_countArray=np.zeros([len(cat),nlum])
    mybin = np.floor((cenmag[:,:]-minlum)/dlum).astype(np.int)
    if weight_cen:
        ncen = np.repeat(cat['ncent_good'].reshape(-1,1),cat['p_cen'].shape[1],axis=1)
        weight = cat['p_cen']
    else:
        ncen = np.zeros(cat['p_cen'].shape)
        ncen = np.hstack(((np.ones(cat['p_cen'].shape[0])).reshape(-1,1),ncen[:,:-1])).astype(np.int) 
        weight = cat['p_sat']+cat['p_cen']
    
    ys = np.outer(np.ones(cat['p_cen'].shape[0]),np.arange(cat['p_cen'].shape[1]))
    mask = ((mybin>=0) & (mybin<nlum)& (ys<ncen)).astype(np.float64)
    newbin=mybin*mask
    #index=np.arange(nclusters)
    indexArray=np.arange(nlum)
    for i in indexArray:
        masknew=(mybin==i).astype(np.float64)*mask
        newNewbin=np.sum(masknew*weight,axis=1)
        chto_countArray[:,i]=newNewbin[:]
    return chto_countArray

In [39]:
#%time chto_countArray=chto_count_galaxies_rand_cen(cat,cenmag,cengalindex,lumbins,weight_cen=0)

In [42]:
#%time countarr_weight=count_galaxies_rand_cen(cat,cenmag,cengalindex,lumbins,weight_cen=1)
#%time chto_countArray_weight=chto_count_galaxies_rand_cen(cat,cenmag,cengalindex,lumbins,weight_cen=1)
#%time countarr_weight=count_galaxies_p_cen(cat,cenmag,lumbins,weight_cen=0)
%time chto_countArray_weight=chto_count_galaxies_p_cen(cat,cenmag,lumbins,weight_cen=0)

CPU times: user 105 ms, sys: 25 ms, total: 130 ms
Wall time: 97.8 ms


In [43]:
print countarr_weight[0]
print chto_countArray_weight[0]

[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.99999858  0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.        ]
[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.99999857  0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.        ]


In [50]:
print np.where((countarr_weight[:nclusters]-chto_countArray_weight[:nclusters])>=1E-7)

(array([], dtype=int64), array([], dtype=int64))
